In [67]:
import numpy as np
import cv2
import math
import scipy.signal

def harrisCorners(im, sigma):
    dim = np.ceil(sigma * 4)
    if dim % 2 != 0:
        dim+=1
    
    oneMat =  np.ones((int(dim/2), int(dim/2)), dtype=int)
    negMat = -1 * oneMat
    D_x = np.hstack((negMat, oneMat))
    D_x = np.vstack((D_x, D_x))
    D_y = np.vstack((oneMat, negMat))
    D_y = np.hstack((D_y, D_y))
    
    dx = cv2.filter2D(im, -1, D_x)
    dy = cv2.filter2D(im, -1, D_y)
    
    C_size = int(np.ceil(sigma * 5))
    if (C_size % 2 == 0):
        C_size+=1
    C_size_half = int(C_size / 2)
    threshold = np.zeros(im.shape)
    
    for i in range(im.shape[0] - C_size):
        for j in range(im.shape[1] - C_size):
            dx_kernal = dx[i:i+C_size, j:j+C_size]
            dy_kernal = dy[i:i+C_size, j:j+C_size]
            dx2 = np.sum(np.square(dx_kernal))
            dy2 = np.sum(np.square(dy_kernal))
            dxdy = np.sum(np.multiply(dx_kernal, dy_kernal))
            C_det = np.subtract(np.multiply(dx2, dy2), np.square(dxdy))
            C_tr = np.add(dx2, dy2)
            threshold[i + C_size_half, j + C_size_half] = C_det - 0.05 * np.square(C_tr)

    thresh_size = 29
    thresh_size_half = int(thresh_size / 2) 
    thresholdAvg = np.mean(abs(threshold))
    corners=[]
    
    for i in range(im.shape[0]-thresh_size):
        for j in range(im.shape[1]-thresh_size):
            if threshold[i+thresh_size_half,j+thresh_size_half] > 0 and abs(threshold[i+thresh_size_half,j+thresh_size_half]) > thresholdAvg:
                thresh = threshold[i:i+thresh_size,j:j+thresh_size]
                if threshold[i+thresh_size_half,j+thresh_size_half] == np.max(thresh):
                    corners.append([i+thresh_size_half,j+thresh_size_half])
    return np.array(corners)
    
def NCC(im1, im2, cnr1, cnr2):
    NCC = np.zeros((len(cnr1), len(cnr2)))   
    for i in range(len(cnr1)):
        for j in range(len(cnr2)):
            f1 = im1[cnr1[i, 0] - 12 : cnr1[i, 0] + 13, cnr1[i, 1] - 12 : cnr1[i, 1] + 13]
            f2 = im2[cnr2[j, 0] - 12 : cnr2[j, 0] + 13, cnr2[j, 1] - 12 : cnr2[j, 1] + 13]
            ncc = np.sum(np.multiply(np.subtract(f1, np.mean(f1)), np.subtract(f2, np.mean(f2))))
            ncc /= np.sqrt(np.sum(np.square(np.subtract(f1, np.mean(f1)))) * np.sum(np.square(np.subtract(f2, np.mean(f2)))))
            NCC[i, j] = ncc

    pts=[]
    for i in range(len(cnr1)):
        for j in range(len(cnr2)):
            if NCC[i,j]==np.max(NCC[i,:]) and NCC[i,j]>0.8:
                pts.append([[cnr1[i,0],cnr1[i,1]],[cnr2[j,0],cnr2[j,1]]])

    return np.array(pts)

def getHomography(im1Pts, im2Pts):
    px = im1Pts[0, 1]
    py = im1Pts[0, 0]
    qx = im1Pts[1, 1]
    qy = im1Pts[1, 0]
    rx = im1Pts[2, 1]
    ry = im1Pts[2, 0]
    sx = im1Pts[3, 1]
    sy = im1Pts[3, 0]
    
    pxp = im2Pts[0, 1]
    pyp = im2Pts[0, 0]
    qxp = im2Pts[1, 1]
    qyp = im2Pts[1, 0]
    rxp = im2Pts[2, 1]
    ryp = im2Pts[2, 0]
    sxp = im2Pts[3, 1]
    syp = im2Pts[3, 0]
    
    t = np.array([[pxp],[pyp],[qxp],[qyp],[rxp],[ryp],[sxp],[syp]])
    P = np.array([[px, py, 1, 0, 0, 0, -px*pxp, -py*pxp],
                  [0, 0, 0, px, py, 1, -px*pyp, -py*pyp],
                  [qx, qy, 1, 0, 0, 0, -qx*qxp, -qy*qxp],
                  [0, 0, 0, qx, qy, 1, -qx*qyp, -qy*qyp],
                  [rx, ry, 1, 0, 0, 0, -rx*rxp, -ry*rxp],
                  [0, 0, 0, rx, ry, 1, -rx*ryp, -ry*ryp],
                  [sx, sy, 1, 0, 0, 0, -sx*sxp, -sy*sxp],
                  [0, 0, 0, sx, sy, 1, -sx*syp, -sy*syp]])
    pInv = np.linalg.inv(P)
    h = pInv.dot(t)
    H = np.array([[h[0, 0], h[1, 0], h[2, 0]], [h[3, 0], h[4, 0], h[5, 0]], [h[6, 0], h[7, 0], 1]])
    return H

def testHomography(pts, H):
    xp = np.linalg.matmul(H, np.append(pt[0], 1))
    xp /= xp[-1]
    dist = math.sqrt(np.sum((xp[0] - pt[1][0])**2, (xp[1] - pt[1][1])**2))

def RANSAC(pts):   
    pts1 = pts[:0]
    pts2 = pts[:1]
    N = 96
    M = (len(pts)*0.6)
    maxNum = 0
    bestH = np.zeros(3,3)
    inliers = []
    outliers = []
    for i in range(N):
        rand = np.random.randint(0,len(pts),6)
        test1 = pts[0][rand]
        test2 = pts[1][rand]
        rand = np.random.randint(0,len(pts),6)
        test3 = pts[0][rand]
        test4 = pts[1][rand]
        rand = np.random.randint(0,len(pts),6)
        test5 = pts[0][rand]
        test6 = pts[1][rand]
        rand = np.random.randint(0,len(pts),6)
        test7 = pts[0][rand]
        test8 = pts[1][rand]
        Htest = getHomography(np.array([test1, test3, test5, test7]), np.array([test2, test4, test6, test8]))
        num = 0
        tempInliers = []
        tempOutliers = []
        for j in range(pts):
            dist = testHomography(pts[j], Htest)
            if dist < 3:
                num+=1
                tempInliers.append(pts[j])
            else:
                tempOutliers.append(pts[j])
        if num > maxNum:
            maxNum = num
            bestH = Htest
            inliers = tempInliers
            outliers = tempOutliers
    
    return bestH, inliers, outliers

def showPoints(im1, im2, corners):    
    im = np.zeros((max(im1.shape[0], im2.shape[0]), im1.shape[1] + im2.shape[1],3))
    im[0 : im1.shape[0], 0 : im1.shape[1]] = im1
    im[0 : im2.shape[0], im1.shape[1] : im1.shape[1] + im2.shape[1]] = im2
    
    for cnr in corners:
        cv2.circle(im,(cnr[0, 1], cnr[0, 0]),4,(0,255,0),1)
        cv2.circle(im,(im1.shape[1]+cnr[1, 1],cnr[1, 0]),4,(0,255,0),1)
        cv2.line(im,(cnr[0, 1],cnr[0, 0]),(im1.shape[1]+cnr[1, 1],cnr[1, 0]), (0,255,0))
    return im

def showInliers(im1, im2, inliers, outliers):
    im = np.zeros((max(im1.shape[0], im2.shape[0]), im1.shape[1] + im2.shape[1],3))
    im[0 : im1.shape[0], 0 : im1.shape[1]] = im1
    im[0 : im2.shape[0], im1.shape[1] : im1.shape[1] + im2.shape[1]] = im2
    
    for cnr in inliers:
        cv2.circle(im,(cnr[0, 1], cnr[0, 0]),4,(0,255,0),1)
        cv2.circle(im,(im1.shape[1]+cnr[1, 1],cnr[1, 0]),4,(0,255,0),1)
        cv2.line(im,(cnr[0, 1],cnr[0, 0]),(im1.shape[1]+cnr[1, 1],cnr[1, 0]), (0,255,0))
        
    for cnr in outliers:
        cv2.circle(im,(cnr[0, 1], cnr[0, 0]),4,(0,0,255),1)
        cv2.circle(im,(im1.shape[1]+cnr[1, 1],cnr[1, 0]),4,(0,0,255),1)
        cv2.line(im,(cnr[0, 1],cnr[0, 0]),(im1.shape[1]+cnr[1, 1],cnr[1, 0]), (0,0,255))

images = ['Images/1.jpg', 'Images/2.jpg', 'Images/3.jpg', 'Images/4.jpg', 'Images/5.jpg']
im = []; imGray = []; corners = [];
sift = cv2.xfeatures2d.SIFT_create(nfeatures=5000,nOctaveLayers=4,contrastThreshold=0.03,edgeThreshold=10,sigma=4)# may want to add these arguements
for img in images:
    image = cv2.imread(img)
    im.append(image)
    imageGray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    imGray.append(imageGray)
    corners.append(harrisCorners(imageGray, 1.2))

intPts = []
for i in range(4):
    intPts.append(NCC(im[i], im[i+1], corners[i], corners[i + 1]))

cv2.imwrite("Results/12.jpg", showPoints(im[0], im[1], intPts[0]))
cv2.imwrite("Results/23.jpg", showPoints(im[1], im[2], intPts[1]))
cv2.imwrite("Results/34.jpg", showPoints(im[2], im[3], intPts[2]))
cv2.imwrite("Results/45.jpg", showPoints(im[3], im[4], intPts[3]))
    
H12, inliers12, outliers12 = RANSAC(intPts[0])
cv2.imwrite("Results/inliers12.jpg", showInliers(im[0], im[1], inliers12, outliers12))
H23, inliers23, outliers23 = RANSAC(intPts[1])
cv2.imwrite("Results/inliers23.jpg", showInliers(im[1], im[2], inliers23, outliers23))
H34, inliers34, outliers34 = RANSAC(intPts[2])
cv2.imwrite("Results/inliers34.jpg", showInliers(im[2], im[3], inliers34, outliers34))
H45, inliers45, outliers45 = RANSAC(intPts[3])
cv2.imwrite("Results/inliers45.jpg", showInliers(im[3], im[4], inliers45, outliers45))

stitch1 = stitch(im[0], im[1], H12)
stitch2 = stitch(stitch1, im[2], H23)
stitch3 = stitch(stitch2, im[3], H34)
stitch4 = stitch(stitch3, im[4], H45)
cv2.imwrite("Results/Final.jpg", stitch4)

TypeError: data type not understood

In [42]:
print(len(intPts12))
intPts12 = intPts12.astype(int)
cv2.imwrite('out.jpg',showPoints(im[0], im[1], intPts12))

2
loop
loop


True